# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [4]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,lorengau,-2.0226,147.2712,300.67,82,100,2.59,PG,1714760960
1,1,waitangi,-43.9535,-176.5597,282.49,82,91,3.58,NZ,1714760960
2,2,turpan,42.9333,89.1667,297.45,12,76,0.77,CN,1714760960
3,3,kingston,17.9970,-76.7936,303.68,70,40,8.23,JM,1714760753
4,4,nogliki,51.8333,143.1667,271.67,92,26,3.13,RU,1714760960


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
# Configure the map plot
map_1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 700,
    frame_height = 500,
    size = 'Humidity',
    scale = 1,
    alpha = 0.5,
    color = 'City'
)

# Display the map
map_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria
ideal_cities_df = city_data_df[(city_data_df['Max Temp'] > 21) & 
                               (city_data_df['Max Temp'] < 27) &
                               (city_data_df['Wind Speed'] < 4.5) &
                               (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame({
    'City': ideal_cities_df['City'],
    'Country': ideal_cities_df['Country'],
    'Lat': ideal_cities_df['Lat'],
    'Lng': ideal_cities_df["Lng"],
    'Humidity': ideal_cities_df['Humidity'],
    'Hotel Name': ''
    })

hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set parameters to search for a hotel
radius = 10000  # Set the search radius to 10,000 meters
params = {
    "type": "hotel",  # Specify the type of place to search for (hotel)
    "radius": radius,  # Set the search radius
    "limit": 1,  # Limit the number of results to 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add latitude and longitude to the params dictionary
    params["lat"] = lat
    params["lon"] = lng
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    hotel_data = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_data["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
print(hotel_df.head())



Starting hotel search
Empty DataFrame
Columns: [City, Country, Lat, Lng, Humidity, Hotel Name]
Index: []


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
import plotly.express as px

# Create a scatter plot on the map
fig = px.scatter_mapbox(hotel_df, lat="Lat", lon="Lng", hover_name="City",
                        hover_data={"Hotel Name": True, "Country": True},
                        color="Humidity", size="Humidity",
                        color_continuous_scale=px.colors.sequential.Viridis,
                        size_max=30, zoom=1)

# Update layout to add additional information in hover message
fig.update_layout(mapbox_style="open-street-map",
                  mapbox=dict(center=dict(lat=hotel_df["Lat"].mean(), lon=hotel_df["Lng"].mean())),
                  hovermode='closest',
                  margin=dict(l=0, r=0, t=0, b=0))

# Show the map
fig.show()
